In [1]:
%pip install -Uqqq pip --progress-bar off
%pip install -qqq ollama --progress-bar off
%pip install -qqq pathlib --progress-bar off
%pip install -qqq pandas --progress-bar off
%pip install -qqq PyPDF2 --progress-bar off
%pip install -qqq owlready2 --progress-bar off
%pip install -qqq rdflib --progress-bar off
%pip install -qqq langchain-ollama --progress-bar off
%pip install -qqq langchain-community --progress-bar off
%pip install -qqq langchain_community pypdf --progress-bar off
%pip install -qqq langchain-chroma --progress-bar off


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#all the imports
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import ollama
import json
from enum import Enum
MODEL = "llama3.1:8b-instruct-q8_0"

In [3]:
# Load the PDF
pdf_loader = PyPDFLoader(file_path="sensors.pdf")
docs = pdf_loader.load();


In [4]:
# Calculate Average Document Size
def word_count(texts):
  total_words = 0
  total_characters = 0

  for doc in texts:
      content = doc.page_content
      word_count = len(content.split())  # Count words
      char_count = len(content)         # Count characters
      total_words += word_count
      total_characters += char_count

  num_docs = len(texts)
  avg_words = total_words / num_docs if num_docs > 0 else 0
  avg_characters = total_characters / num_docs if num_docs > 0 else 0

  print(f"Average words per document: {avg_words}")
  print(f"Average characters per document: {avg_characters}")

word_count(docs)

Average words per document: 438.2
Average characters per document: 2702.8


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

word_count(splits)

Average words per document: 136.78666666666666
Average characters per document: 843.32


In [6]:
#create embeddings and store in CHROMA database
embeddings = OllamaEmbeddings(model=MODEL)
vector_store = Chroma(
    collection_name="KGLLM",
    embedding_function=embeddings,
)
# Add documents and their embeddings to Chroma (might take some time)
vector_store.add_documents(documents=splits)

['5f69798c-8746-430b-9a41-12de8b4b26c6',
 '3af57f9e-c00c-47f3-b5f3-295534308b32',
 'be6d821c-7b33-4195-8fe4-7c0348aed782',
 '0995a062-6530-4a5c-ad4a-d8e03e81bf6d',
 'fed973ba-b1da-4adc-98b1-e2639a357c3f',
 'b00e2fc4-e7b3-438c-bfcc-ce0588c117f6',
 '7e9766f6-c147-4af7-9c3f-d5c4f1317afd',
 '35a7ccb8-283a-4774-b29f-f5d49abbaded',
 'c627c17a-3f91-4c25-b592-bf1c23dfc22f',
 '113ffd49-79a4-4ef4-b9a4-6d61f395856e',
 '949d4b4f-f26a-43c3-9a16-1398c2fa27d6',
 'a2189861-fe33-4a06-9bca-1f6a99b05f92',
 '8da8fda0-bfe5-4137-a869-0573c34b252b',
 'b11e36b4-494c-4b28-8e2e-924d09bf6f0d',
 '862179f6-46d4-4a1c-93b6-3ab40a5cf266',
 '270b982d-805b-449b-a0b8-8d7cd999f5a9',
 '6c8d581a-cea5-412f-9921-046e488c00e1',
 '33a6425f-65eb-4f7c-9c26-4c208a21208a',
 '2f54bf12-8d03-40e0-bed4-5c739afd514b',
 'ce053794-840e-48ca-aa1a-14ea37a730a9',
 '47e7b000-081b-4bfc-a84f-2bafb74ce882',
 'b8602ad4-fa50-4daa-8464-9e668f84a0e7',
 '26cc7b74-3870-4241-8119-b91d49975e5b',
 'c1b07ad7-8d5f-491e-a1f8-eac119efea8f',
 '333b5e69-3e95-

In [7]:
#query the database
query = """
Extract different LiDAR sensors, their categories, and detailed parts from the given text. 
Additionally, identify and describe the relationships between each sensor and its parts, 
including any specific functions or configurations of the parts in relation to the sensors.
Also, extract any specific properties (e.g., range, resolution, field of view) and implemented technologies 
or standards (e.g., GNSS, INS) associated with the sensors.
"""

retrieved_docs = vector_store.similarity_search(query, k=15)
# Concatenate the retrieved chunks
retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
# #save the retrieved text
# with open("retrieved_text.txt", "w", encoding="utf-8") as f:
#     f.write(retrieved_text)

In [48]:
SUMMARIZE_PROMPT = """
As a LiDAR sensor expert, your task is to extract, categorize, and group all the **real, specific, and branded** LiDAR sensor names, their associated components (parts), and relevant properties or implementations mentioned in the following text.

**Output Requirements**:
1. Provide a concise and structured list of LiDAR sensors and their associated parts.
2. Extract and include any **specific properties** or **capabilities** (e.g., "range", "resolution", "field of view") for each sensor, using the "has property" relationship.
3. Identify and include any **implemented technologies or standards** (e.g., "GNSS", "INS") using the "implements" relationship.
4. Group and standardize similar categories to avoid duplicates (e.g., "Automotive LiDAR", "Automotive LiDAR Sensors", and "Automotive Sensors" should be grouped under one category, "Automotive LiDAR").
5. Categorize the sensors based on their application, type, or any mentioned specifications (e.g., Automotive, Industrial, Surveying).
6. Identify and list the **specific parts or components** of each sensor (e.g., "Laser Module", "Spinning Mechanism", "Control Unit"). If no parts are mentioned for a sensor, leave the "parts" field empty.

**Exclusion Criteria**:
- Do not include generic terms such as "LiDAR", "LiDAR sensor", "LiDAR sensors", "various LiDAR sensors", or "no specific brand model mentioned".
- Exclude datasets or evaluation tools (e.g., "Kitti", "NuScenes").
- Ignore descriptions or summaries like "solid-state LiDAR", "lidar technology", or "selected sensors are mechanical or solid-state types".
- Avoid placeholder entries like "not mentioned" or "various".

**Formatting**:
- Strictly follow the JSON template below and ensure no additional text or explanations are provided:
  {{
    "sensors": [
      {{
        "name": "Velodyne HDL-64E",
        "category": "Automotive LiDAR",
        "parts": ["Laser Module", "Spinning Mechanism"],
        "properties": ["range: 100m", "resolution: 0.2 degrees"],
        "implements": ["GNSS", "INS"]
      }}
    ]
  }}

**Additional Instructions**:
- Only include a category if it contains sensors.
- Use consistent naming conventions for categories. Avoid redundancy or variations in names.
- If no sensors, parts, properties, or implementations are found in the text, return an empty JSON: {{"sensors": []}}.
- Do not include any additional explanations or information; only return the JSON output.

<text>
{text}
</text>
"""


In [49]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"
 
 
def call_model(
    prompt: str, response_format: ResponseFormat = ResponseFormat.TEXT
) -> str:
    response = ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        format="" if response_format == ResponseFormat.TEXT else "json",
    )
    return response["response"]

In [50]:
# Prepare the final prompt with the concatenated text

responses = []
CHUNK_SIZE = 1000
chunks = [retrieved_text[i:i + CHUNK_SIZE] for i in range(0, len(retrieved_text), CHUNK_SIZE)];
for i, chunk in enumerate(chunks):
        final_prompt = SUMMARIZE_PROMPT.format(text=chunk)
    # Send the final prompt to Ollama
        response = call_model(final_prompt)
        responses.append(response)
print(responses)


['{\n  "sensors": [\n    {\n      "name": "Velodyne HDL-64E",\n      "category": "Automotive LiDAR",\n      "parts": ["Laser Module", "Spinning Mechanism"],\n      "properties": ["range: 100m", "resolution: 0.2 degrees"],\n      "implements": ["GNSS", "INS"]\n    },\n    {\n      "name": "Quanergy M8",\n      "category": "Industrial LiDAR",\n      "parts": ["Laser Module", "Spinning Mechanism"],\n      "properties": ["range: 100m", "resolution: 0.1 degrees"],\n      "implements": []\n    },\n    {\n      "name": "Ibeo LUX",\n      "category": "Automotive LiDAR",\n      "parts": ["Laser Module", "Spinning Mechanism"],\n      "properties": ["range: 200m", "resolution: 0.1 degrees"],\n      "implements": ["GNSS"]\n    },\n    {\n      "name": "LeddarTech M16",\n      "category": "Industrial LiDAR",\n      "parts": ["Laser Module", "Spinning Mechanism"],\n      "properties": ["range: 150m", "resolution: 0.2 degrees"],\n      "implements": []\n    }\n  ]\n}', '{\n  "sensors": [\n    {\n    

In [51]:
# Save responses after every chunk to ensure progress is retained
with open("responses.json", "w", encoding="utf-8") as f:
    json.dump(responses, f)

In [52]:
import json
import unicodedata
import re

# Function to clean and normalize strings
def clean_string(s):
    # s = s.strip()  # Remove leading/trailing whitespace
    # s = s.encode('utf-8').decode('unicode_escape')  # Decode Unicode escapes
    # s = unicodedata.normalize('NFKC', s)  # Normalize special characters
    # s = re.sub(r"[\s\_/]+", " ", s)  # Replace special separators with space
    return s.lower()

# Load responses and process them
with open("responses.json", "r", encoding="utf-8") as f:
    responses = json.load(f)

# Placeholder for final results and a set to track unique sensors
final_sensors = []
seen_sensors = set()

# Process responses into final_sensors
for response in responses:
    try:
        response_json = json.loads(response)

        if "sensors" in response_json and isinstance(response_json["sensors"], list):
            for sensor in response_json["sensors"]:
                sensor_name = clean_string(sensor.get("name", "Unnamed Sensor"))
                category_name = clean_string(sensor.get("category", "Uncategorized"))
                parts_list = sensor.get("parts", [])
                properties_list = sensor.get("properties", [])
                implements_list = sensor.get("implements", [])

                # Clean and sort parts to ensure consistent order
                cleaned_parts = sorted(clean_string(part) for part in parts_list)
                cleaned_properties = sorted(clean_string(prop) for prop in properties_list)
                cleaned_implements = sorted(clean_string(imp) for imp in implements_list)
               
                # Create a tuple to represent the sensor uniquely
                sensor_tuple = (sensor_name, category_name, tuple(cleaned_parts), tuple(cleaned_properties), tuple(cleaned_implements))

                # If the combination is not seen, add it to the list
                if sensor_tuple not in seen_sensors:
                    seen_sensors.add(sensor_tuple)
                    final_sensors.append({
                        "name": sensor_name,
                        "category": category_name,
                        "parts": cleaned_parts,
                        "properties": cleaned_properties,
                        "implements": cleaned_implements
                    })
    except json.JSONDecodeError as e:
        pass
        # print(f"Error parsing response: {response}. Details: {e}")

# Save the cleaned and unique sensors to a new file
with open("final_sensors_with_parts5.json", "w", encoding="utf-8") as out_file:
    json.dump({"sensors": final_sensors}, out_file, ensure_ascii=False, indent=4)

print("Sensors processed, duplicates removed, and saved to final_sensors_with_parts5.json.")


Sensors processed, duplicates removed, and saved to final_sensors_with_parts5.json.


In [53]:
from owlready2 import *
import json

# Load the JSON file
with open("final_sensors_with_parts5.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Create a new ontology
onto = get_ontology("http://example.org/sensor_ontology.owl")

with onto:
    # Define basic classes and properties
    class Sensor(Thing):
        pass
    
    class Part(Thing):
        pass
    
    class Property(Thing):
        pass
    
    class Technology(Thing):
        pass

    class has_part_directly(ObjectProperty):
        domain = [Sensor]
        range = [Part]
    
    class implements(ObjectProperty):
        domain = [Sensor]
        range = [Technology]
    
    class has_property(ObjectProperty):
        domain = [Sensor]
        range = [Property]

    # Define an annotation property for category
    class category(AnnotationProperty):
        pass

    # Process the JSON data
    part_classes = {}  # Cache for part classes to avoid duplicates
    property_classes = {}  # Cache for property classes
    technology_classes = {}  # Cache for technology classes

    for sensor_data in data["sensors"]:
        # Create or get the sensor class
        sensor_name = sensor_data["name"].replace(" ", "_")
        sensor_category = sensor_data["category"].replace(" ", "_")
        
        # Define the sensor class dynamically
        sensor_class = types.new_class(sensor_name, (Sensor,))
        
        # Assign category as an annotation
        sensor_class.category.append(sensor_category)

        # Add parts and relationships
        for part_name in sensor_data["parts"]:
            part_name_cleaned = part_name.replace(" ", "_")
            if part_name_cleaned not in part_classes:
                part_class = types.new_class(part_name_cleaned, (Part,))
                part_classes[part_name_cleaned] = part_class
            else:
                part_class = part_classes[part_name_cleaned]

            # Create the relationship
            sensor_class.is_a.append(has_part_directly.some(part_class))

        # Add properties and relationships
        for property_name in sensor_data["properties"]:
            property_name_cleaned = property_name.replace(" ", "_")
            if property_name_cleaned not in property_classes:
                property_class = types.new_class(property_name_cleaned, (Property,))
                property_classes[property_name_cleaned] = property_class
            else:
                property_class = property_classes[property_name_cleaned]

            # Create the relationship
            sensor_class.is_a.append(has_property.some(property_class))

        # Add technologies (implements relationship)
        for tech_name in sensor_data["implements"]:
            tech_name_cleaned = tech_name.replace(" ", "_")
            if tech_name_cleaned not in technology_classes:
                tech_class = types.new_class(tech_name_cleaned, (Technology,))
                technology_classes[tech_name_cleaned] = tech_class
            else:
                tech_class = technology_classes[tech_name_cleaned]

            # Create the relationship
            sensor_class.is_a.append(implements.some(tech_class))

# Save the ontology to a file
onto.save(file="sensor_ontology_with_properties_and_implements.owl", format="rdfxml")

print("Ontology created and saved as sensor_ontology_with_properties_and_implements.owl")


Ontology created and saved as sensor_ontology_with_properties_and_implements.owl


In [14]:

# Load the ontology
onto = get_ontology("sensor_ontology.owl").load()

# Print basic information to check loading
print(onto.base_iri)
print(list(onto.classes()))
print(list(onto.properties()))

http://example.org/sensor_ontology.owl#
[sensor_ontology.Sensor, sensor_ontology.Part, sensor_ontology.velodyne_velarray, sensor_ontology.robosense_m1, sensor_ontology.livox_horizon, sensor_ontology.horizon_m1_cube, sensor_ontology.cube_range, sensor_ontology.h800_pro, sensor_ontology.velodyne_hdl-64e, sensor_ontology.laser_module, sensor_ontology.spinning_mechanism, sensor_ontology.blickfeld_cube, sensor_ontology.innoviz_pro, sensor_ontology.velarray, sensor_ontology.blickfeld_cube_range]
[sensor_ontology.has_part_directly, sensor_ontology.category]
